# Cargar Imports y libreria de happytransformer

In [ ]:
pip install happytransformer

In [2]:
from happytransformer import HappyGeneration
from happytransformer import GENSettings
from happytransformer import HappyQuestionAnswering
import pandas as pd

# Carga del modelo GPT-NEO
Por ahora estuve cargando el modelo de 125M ya que el modelo de NEOX pide mucho mas recursos y puede que el colab te termine usando todos los recursos, Asi que mejor cargue el modelo de 125M prompts para que colab no se cuelgue

In [3]:
gpt = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")
gptQuestions = HappyQuestionAnswering("GPT-NEO", "EleutherAI/gpt-neo-125M")

Some weights of GPTNeoForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Generacion de preguntas
aún se debe de dar la configuracion al gptneo para que no haya repeticion de preguntas.

In [ ]:
generate_question = gptQuestions.answer_question("four plus four equal eight", "how much is 4+4?",top_k=5)
print(generate_question)
print(generate_question[0].answer)

[QuestionAnsweringResult(answer=' four equal', score=0.0885031670331955, start=9, end=20), QuestionAnsweringResult(answer=' four equal eight', score=0.0769476369023323, start=9, end=26), QuestionAnsweringResult(answer=' plus four equal', score=0.05782534182071686, start=4, end=20), QuestionAnsweringResult(answer=' eight', score=0.05418120697140694, start=20, end=26), QuestionAnsweringResult(answer=' plus four equal eight', score=0.050275303423404694, start=4, end=26)]
 four equal


In [ ]:
generate_question = gptQuestions.answer_question("20*100=2000.", "Can you explain how to calculate 20*1000",top_k=1)
print(generate_question)
print(generate_question[0].answer)

[QuestionAnsweringResult(answer='2000', score=0.04837531968951225, start=7, end=11)]
2000


# GENSettings
Acá genero las opciones para agregarle a la consulta
https://happytransformer.com/text-generation/settings/

## generacion texto

In [ ]:
args= GENSettings(no_repeat_ngram_size=2, max_length=100)

In [ ]:
result = gpt.generate_text("Today we are gonna celebrate ", args=args)
print(result.text)

In [ ]:
args_top_k = GENSettings(no_repeat_ngram_size=2, do_sample=True,top_k=50, temperature=0.7)

In [ ]:
result = gpt.generate_text("Today we are gonna celebrate", args=args_top_k)
print(result.text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 the end of the year in all of our fields, what we've all been waiting for, and what is going to happen with the world.

We all have things to celebrate, but we also have a lot of work to do so we


## Generacion Pregunta
Intente ver si se podia ingresar argumentos tambien al de pregunta pero no funco.

In [ ]:
args_questions = GENSettings(top_k=90,top_p=3.0, temperature= 0.8, no_repeat_ngram_size=1)

##### Intento de crear una pregunta mas hacia matemáticas

In [ ]:
generate_question = gptQuestions.answer_question("First, we multiply a degree-$1$ term and a degree-$2$ term, so we have a degree-$3$ polynomial. We subtract a constant times a degree-$3$ polynomial, so we can have at most a degree-$3$ polynomial, so at most $4$ terms. However, we are unsure if any of the terms will subtract to zero, so we must multiply out the polynomials:  \\begin{align*}\n&(x+4)(2x^2+3x+9)-3(x^3-2x^2+7x)\\\\\n&\\qquad=x(2x^2+3x+9)+4(2x^2+3x+9)-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3+3x^2+9x+8x^2+12x+36-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3+11x^2+21x+36-(3x^3-6x^2+21x)\\\\\n&\\qquad=2x^3-3x^3+11x^2+6x^2+21x-21x+36\\\\\n&\\qquad=-x^3+17x^2+36.\n\\end{align*}As we can see, the linear term drops out, and we are left with $\\boxed{3}$ terms.", "How many nonzero terms are in the expansion of $$(x+4)(2x^2+3x+9)-3(x^3-2x^2+7x)?",top_k=2)
print(generate_question)
print(generate_question[0].answer)

[QuestionAnsweringResult(answer=' unsure if any of the terms will subtract to zero', score=0.0004399297176860273, start=235, end=284), QuestionAnsweringResult(answer=' if any of the terms will subtract to zero', score=0.00036469308543018997, start=242, end=284)]
 unsure if any of the terms will subtract to zero


# Carga de json
Acá voy a estar cargando el archivo resultado2.json para trabajarlo en un pandas para luego cargarlo.

In [27]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    json=fn
mp = pd.read_json(json)


Saving 0.json to 0.json


In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    csv=fn
mp2 = pd.read_csv(csv, header=None)


Saving resultado2.csv to resultado2.csv


In [28]:
mp

,context,question,answer_text,answer_start
0,Resuelve la siguiente ecuación: 2x + 5 = 11,¿Cuál es el valor de x?,"Para resolver esta ecuación, primero restamos ...",33


In [30]:
mp.to_csv('/content/sample_data/0.csv', index=False)

In [18]:
data = [
    ["Resuelve la siguiente ecuación: 2x + 5 = 11", "¿Cuál es el valor de x?", "Para resolver esta ecuación, primero restamos 5 de ambos lados: 2x = 6. Luego, dividimos ambos lados por 2: x = 3.", 33],
    ["Factoriza el siguiente polinomio: x^2 - 4", "¿Cuáles son los factores de este polinomio?", "Para factorizar x^2 - 4, utilizamos la diferencia de cuadrados: (x + 2)(x - 2).", 28],
    ["Encuentra el valor de la variable y en el sistema de ecuaciones: 3x + 2y = 10 y 2x - y = 4", "¿Cuál es el valor de y?", "Para encontrar el valor de y, puedes resolver el sistema de ecuaciones y encontrar que y = 2.", 37],
    ["Resuelve la siguiente inecuación: 3x + 7 > 22", "¿Cuál es el conjunto de soluciones para x?", "Para resolver la inecuación, primero restamos 7 de ambos lados: 3x > 15. Luego, dividimos ambos lados por 3: x > 5.", 0]
]

column_names = ["context", "question", "answer_text", "answer_start"]

df = pd.DataFrame(data, columns=column_names)
df

,context,question,answer_text,answer_start
0,Resuelve la siguiente ecuación: 2x + 5 = 11,¿Cuál es el valor de x?,"Para resolver esta ecuación, primero restamos ...",33
1,Factoriza el siguiente polinomio: x^2 - 4,¿Cuáles son los factores de este polinomio?,"Para factorizar x^2 - 4, utilizamos la diferen...",28
2,Encuentra el valor de la variable y en el sist...,¿Cuál es el valor de y?,"Para encontrar el valor de y, puedes resolver ...",37
3,Resuelve la siguiente inecuación: 3x + 7 > 22,¿Cuál es el conjunto de soluciones para x?,"Para resolver la inecuación, primero restamos ...",0


In [19]:
df = pd.DataFrame(data, columns=column_names)

# Guardar el DataFrame en un archivo CSV
df.to_csv('/content/sample_data/df_mo.csv', index=False)

In [ ]:
from happytransformer import HappyQuestionAnswering, QATrainArgs
# --------------------------------------#
happy_qa_albert = HappyQuestionAnswering("ALBERT", "mfeb/albert-xxlarge-v2-squad2")
args = QATrainArgs(num_train_epochs=1)

In [ ]:
happy_qa_albert.train('/content/sample_data/0.csv')

In [ ]:
mp_cambiado= mp.rename(columns={"problem":"question","level":"answer_start","type":"context","solution":"answer_text"})
mp_ordenado = mp_cambiado[["context","question","answer_text","answer_start"]]

In [ ]:
mp_ordenado['answer_start'] = mp_ordenado['answer_start'].str.extract(r'(\d+)').astype(int)

In [ ]:
mp_ordenado

,context,question,answer_text,answer_start
0,Algebra,How many vertical asymptotes does the graph of...,The denominator of the rational function facto...,3
1,Algebra,What is the positive difference between $120\%...,One hundred twenty percent of 30 is $120\cdot3...,1
2,Algebra,Find $x$ such that $\lceil x \rceil + x = \dfr...,"First, we note that $x$ must be positive, sinc...",4
3,Algebra,Evaluate $i^5+i^{-25}+i^{45}$.,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. ...,5
4,Algebra,"If $2^8=4^x$, what is the value of $x$?",Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...,1
...,...,...,...,...
1182,Algebra,"When Frederick was born, his grandparents gave...","If the interest is simple, then Frederick's mo...",4
1183,Algebra,Simplify the expression $$(x^5+3x^2+3x^5)-(x^7...,"Combining like terms, we find that \begin{ali...",4
1184,Algebra,"A line goes through point $A(9, 1)$, point $B(...",The slope of the line is $\frac{1-0}{9-7}=\fra...,3
1185,Algebra,If $x = 1 + \frac{x}{1 + \frac{x}{1+ \frac{x}{...,Notice that the in the equation $$x = 1 + \fra...,3


In [ ]:
from happytransformer import HappyQuestionAnswering, QATrainArgs
# --------------------------------------#
happy_qa = HappyQuestionAnswering()

args = QATrainArgs(num_train_epochs=1)
happy_qa.train('/content/mp_ordenado.csv', args=args)


############################################################################################

# Hasta el momento me rendi de estar probando con HappyTransformers y comenze a usar las librerias de Pytorch y transformers de HugginFace

In [1]:
!pip install torch

In [4]:
from transformers import pipeline

In [5]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')

In [6]:
prompt = "Today is 10-31-2023 and today is "

In [7]:
res = generator(prompt, max_length=50, do_sample=True, temperature=0.9)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
print(res[0]['generated_text'])

Today is 10-31-2023 and today is 
the date of the first meeting of the Board of Directors of 
Cities in the State of Maryland.               


In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoTokenizer, GPTNeoForQuestionAnswering
import torch
model = GPTNeoForQuestionAnswering.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

Some weights of GPTNeoForQuestionAnswering were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
question, text = "Who was Jimi hendrix?", "Jimi Hendrix was a groundbreaking and influential American rock guitarist and singer known for his virtuosic playing and innovative approach to music."

In [42]:
inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index ]
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

predicted_answer = tokenizer.decode(predict_answer_tokens)
print("Respuesta predicha:", predicted_answer)

Respuesta predicha: ?Jimi Hendrix was a groundbreaking and influential American rock guitarist and singer


In [35]:
print(inputs)

{'input_ids': tensor([[ 8241,   373,  5395,    72,   339,   358,  8609,    30, 18050,    72,
         14666,  8609,   373,   257, 31088,   290, 14212,  1605,  3881, 32705,
           290, 14015,  1900,   329,   465,  4118,    84,   418,   291,  2712,
           290, 13097,  3164,   284,  2647,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
